In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import shap
import logging
import sys
import os
import soundfile as sf
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from datasets import load_dataset
import librosa
import librosa.display
from scipy.stats import pearsonr
from typing import List, Dict, Tuple
import json
from pathlib import Path
from tqdm import tqdm
import warnings

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout),
        logging.FileHandler('evaluation.log')
    ]
)
logger = logging.getLogger(__name__)

model_name = "facebook/wav2vec2-base-960h"
            

/home/hagen/SHAP-Transformer-ASR/.shaptransformerasr/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-12 22:41:26.788600: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-12 22:41:26.800107: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757709686.813789    8973 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757709686.817920    8

In [2]:
def _create_model_wrapper(model):
    """Create a wrapper class for the model to get properly shaped output"""
    class ModelWrapper(torch.nn.Module):
        def __init__(self, model):
            super().__init__()
            self.model = model
        
        def forward(self, x):
            # Ensure input has correct shape for the model
            if len(x.shape) == 4:
                x = x.squeeze(1).squeeze(1)  # Remove extra dimensions
            elif len(x.shape) == 3:
                x = x.squeeze(1)  # Remove extra dimension
            
            # Add attention mask
            attention_mask = torch.ones_like(x)
            
            # Forward pass with attention mask
            logits = self.model(x, attention_mask=attention_mask).logits
            
            # Log the shape and statistics of logits
            logger.debug(f"Logits shape: {logits.shape}")
            logger.debug(f"Logits mean: {torch.mean(logits).item():.6f}")
            logger.debug(f"Logits std: {torch.std(logits).item():.6f}")
            
            # For SHAP, aggregate over vocab to get a scalar per time step
            return torch.max(logits,dim=-1).values  # [batch, seq_len]
    
    return ModelWrapper(model)

In [3]:
def _add_noise(audio: np.ndarray, snr_db: float) -> np.ndarray:
    """Add white noise to audio at specified SNR"""
    signal_power = np.mean(audio ** 2)
    noise_power = signal_power / (10 ** (snr_db / 10))
    noise = np.random.normal(0, np.sqrt(noise_power), len(audio))
    return audio + noise

In [4]:
def create_test_set(num_samples: int = 10) -> Dict:
    """Create a controlled test set with various conditions"""
    logger.info(f"Creating test set with {num_samples} samples")
    ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")
    test_set = []

    for i in tqdm(range(min(num_samples, len(ds))), desc="Creating test samples"):
        sample = ds[i+4]
        audio = sample["audio"]["array"]
        if len(audio) > 100000:
            continue  # Skip too long samples
        text = sample["text"]
        
        # Create clean sample
        test_set.append({
            "type": "clean",
            "audio": audio,
            "text": text,
            "snr": float('inf'),
            "noise": np.zeros_like(audio)
        })
        logger.info(f"Added clean sample {i+1}")
        
        # Create noisy samples with different SNRs [20, 10, 0, -5]
        for snr in tqdm([5,2,1], desc=f"Adding noise to sample {i+1}", leave=False):
            noisy_audio = _add_noise(audio, snr)
            test_set.append({
                "type": "noisy",
                "audio": noisy_audio,
                "text": text,
                "snr": snr,
                "noise": noisy_audio - audio
            })
            logger.info(f"Added noisy sample {i+1} with SNR {snr}dB")

    logger.info(f"Test set created with {len(test_set)} total samples")
    return test_set

In [5]:
def compute_shap_values(processor, device, wrapped_model, model, vocab, audio: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """Compute SHAP values for an audio sample using GradientExplainer"""
    logger.info("Computing SHAP values")
    # Process audio
    inputs = processor(audio, sampling_rate=16000, return_tensors="pt")
    input_values = inputs.input_values.to(device)
    
    # Ensure input_values has the correct shape [batch_size, sequence_length]
    if len(input_values.shape) == 3:
        input_values = input_values.squeeze(1)
    
    # Create background samples with correct shape [batch_size, sequence_length]
    num_background = 5
    background = torch.zeros((num_background, input_values.shape[1]), device=device)
    background += torch.randn_like(background) * 0.01  # Add small random noise
    logger.info(f"Created background samples with shape {background.shape}")
    logger.info(f"Background mean: {torch.mean(background).item():.6f}")
    logger.info(f"Background std: {torch.std(background).item():.6f}")
    
    # Initialize GradientExplainer with the model only
    explainer = shap.GradientExplainer(
        wrapped_model,
        background,
        batch_size=1
    )
    
    # Log model output before SHAP computation
    with torch.no_grad():
        model_output = wrapped_model(input_values)
        logger.info(f"Model output shape: {model_output.shape}")
        logger.info(f"Model output mean: {torch.mean(model_output).item():.6f}")
        logger.info(f"Model output std: {torch.std(model_output).item():.6f}")
        logger.info(f"Model output sum: {torch.sum(model_output).item():.6f}")
    
    # retrieve logits
    logits = model(input_values).logits

    # take argmax and decode
    predicted_ids = torch.argmax(logits, dim=-1)
    logger.info(f"Predicted IDs: {predicted_ids}")
    transcription = processor.batch_decode(predicted_ids)
    logger.info(f"Transcription: {transcription}")

    output_string = ''.join([list(vocab.keys())[list(vocab.values()).index(id.item())] for id in predicted_ids[0]])
    logger.info(f"Decoded output string: {output_string}")

    # Get SHAP values
    logger.info("Computing SHAP values with GradientExplainer")
    shap_values = explainer.shap_values(input_values)
    logger.info(f"Raw SHAP values type: {type(shap_values)}")
    
    # Convert to numpy and process
    # if isinstance(shap_values[0], torch.Tensor):
    #     shap_values = [v.cpu().numpy() for v in shap_values]
    
    # Convert to numpy array and handle shapes
    # shap_values = np.array(shap_values)  # Shape: (1, batch, seq_len)
    logger.info(f"SHAP values shape after conversion: {shap_values.shape}")
    
    return shap_values

In [6]:
def compute_metrics(processor, device, wrapped_model, model, vocab, test_set: List[Dict]) -> Dict:
    """Compute evaluation metrics for the test set"""
    logger.info("Computing metrics for test set")
    metrics = {
        "shap_noise_correlation": [],
        "shap_confidence_correlation": [],
        "wer_correlation": []
    }
    
    # Store SHAP values and other computed values for visualization
    visualization_data = []
    
    for i, sample in enumerate(tqdm(test_set, desc="Computing metrics")):
        logger.info(f"Processing sample {i+1}/{len(test_set)}")
        audio = sample["audio"]
        text = sample["text"]
        
        # Get model prediction and confidence
        inputs = processor(audio, sampling_rate=16000, return_tensors="pt")
        input_values = inputs.input_values.to(device)
        
        with torch.no_grad():
            logits = model(input_values).logits
            probs = torch.softmax(logits, dim=-1)
            confidence = torch.mean(torch.max(probs, dim=-1)[0]).item()
        logger.info(f"Model confidence: {confidence:.4f}")
        
        # Compute SHAP values
        shap_values = compute_shap_values(processor, device, wrapped_model, model, vocab, audio)
        logger.info(f"SHAP values shape: {shap_values.shape}")
        logger.info(f"SHAP values range: [{np.min(shap_values):.4f}, {np.max(shap_values):.4f}]")

        np.save(f"data/shap_values_sample_{i+1+13}_{sample['type']}_{sample['snr']}",shap_values)
        np.save(f"data/audio_sample_{i+1+13}_{sample['type']}_{sample['snr']}",sample["audio"])
        np.save(f"data/noise_sample_{i+1+13}_{sample['type']}_{sample['snr']}",sample["noise"])

In [7]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logger.info(f"Using device: {device}")

# Load model and processor
logger.info(f"Loading model: {model_name}")
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)
model = model.to(device)
vocab = {"<pad>": 0, "<s>": 1, "</s>": 2, "<unk>": 3, "|": 4, "E": 5, "T": 6, "A": 7, "O": 8, "N": 9, "I": 10, "H": 11, "S": 12, "R": 13, "D": 14, "L": 15, "U": 16, "M": 17, "W": 18, "C": 19, "F": 20, "G": 21, "Y": 22, "P": 23, "B": 24, "V": 25, "K": 26, "'": 27, "X": 28, "J": 29, "Q": 30, "Z": 31}
logger.info("Model loaded successfully")

# Create model wrapper for SHAP
wrapped_model = _create_model_wrapper(model)
logger.info("Model wrapper created")

2025-09-12 22:41:32,656 - INFO - Using device: cuda
2025-09-12 22:41:32,657 - INFO - Loading model: facebook/wav2vec2-base-960h


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


2025-09-12 22:41:33,938 - INFO - Model loaded successfully
2025-09-12 22:41:33,939 - INFO - Model wrapper created


In [8]:
# Initialize evaluator


# Create test set
logger.info("Creating test set...")
test_set = create_test_set(num_samples=20)
logger.info(test_set)

2025-09-12 22:41:36,513 - INFO - Creating test set...
2025-09-12 22:41:36,514 - INFO - Creating test set with 20 samples


Creating test samples:   5%|███                                                          | 1/20 [00:00<00:16,  1.13it/s]

2025-09-12 22:41:38,765 - INFO - Added clean sample 3


2025-09-12 22:41:38,770 - INFO - Added noisy sample 3 with SNR 5dB
2025-09-12 22:41:38,773 - INFO - Added noisy sample 3 with SNR 2dB
2025-09-12 22:41:38,775 - INFO - Added noisy sample 3 with SNR 1dB


2025-09-12 22:41:38,782 - INFO - Added clean sample 5


...

2025-09-12 22:41:38,890 - INFO - Added noisy sample 19 with SNR 5dB
2025-09-12 22:41:38,891 - INFO - Added noisy sample 19 with SNR 2dB
2025-09-12 22:41:38,893 - INFO - Added noisy sample 19 with SNR 1dB


2025-09-12 22:41:38,897 - INFO - Added clean sample 20


2025-09-12 22:41:38,900 - INFO - Added noisy sample 20 with SNR 5dB
2025-09-12 22:41:38,902 - INFO - Added noisy sample 20 with SNR 2dB
2025-09-12 22:41:38,904 - INFO - Added noisy sample 20 with SNR 1dB


Creating test samples: 100%|████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 19.46it/s]

2025-09-12 22:41:38,906 - INFO - Test set created with 44 total samples
2025-09-12 22:41:38,906 - INFO - [{'type': 'clean', 'audio': array([ 0.00030518,  0.00030518,  0.00018311, ...,  0.        ,
       -0.00091553,  0.00366211]), 'text': 'ON THE GENERAL PRINCIPLES OF ART MISTER QUILTER WRITES WITH EQUAL LUCIDITY', 'snr': inf, 'noise': array([0., 0., 0., ..., 0., 0., 0.])}, {'type': 'noisy', 'audio': array([-0.04837483,  0.0246488 , -0.00958223, ..., -0.02770247,
       -0.00319669,  0.00140494]), 'text': 'ON THE GENERAL PRINCIPLES OF ART MISTER QUILTER WRITES WITH EQUAL LUCIDITY', 'snr': 5, 'noise': array([-0.04868   ,  0.02434363, -0.00976534, ..., -0.02770247,
       -0.00228116, -0.00225717])}, {'type': 'noisy', 'audio': array([-0.09383595, -0.0114319 , -0.04057077, ...,  0.00613443,
       -0.02603626, -0.01330276]), 'text': 'ON THE GENERAL PRINCIPLES OF ART MISTER QUILTER WRITES WITH EQUAL LUCIDITY', 'snr': 2, 'noise': array([-0.09414112, -0.01173708, -0.04075387, ...,  0.006134

In [19]:
print(len(test_set))
sample = test_set[0]
np.save(f"data/audio_sample_{0}_{sample['type']}_{sample['snr']}",sample["audio"])
audio = np.load("data/audio_sample_0_clean_inf.npy")
sf.write("data/test_audio.wav",audio, 16000)

44


In [9]:
# Compute metrics and get visualization data
logger.info("Computing metrics...")
compute_metrics(processor, device, wrapped_model, model, vocab, test_set)

2025-09-12 22:41:47,227 - INFO - Computing metrics...
2025-09-12 22:41:47,228 - INFO - Computing metrics for test set


Computing metrics:   0%|                                                                         | 0/44 [00:00<?, ?it/s]

2025-09-12 22:41:47,230 - INFO - Processing sample 1/44
2025-09-12 22:41:47,586 - INFO - Model confidence: 0.9816
2025-09-12 22:41:47,587 - INFO - Computing SHAP values
2025-09-12 22:41:47,610 - INFO - Created background samples with shape torch.Size([5, 90240])
2025-09-12 22:41:47,612 - INFO - Background mean: 0.000008
2025-09-12 22:41:47,614 - INFO - Background std: 0.010011
2025-09-12 22:41:48,079 - INFO - Model output shape: torch.Size([1, 281])
2025-09-12 22:41:48,080 - INFO - Model output mean: 13.266081
2025-09-12 22:41:48,081 - INFO - Model output std: 1.645514
2025-09-12 22:41:48,092 - INFO - Model output sum: 3727.768799
2025-09-12 22:41:48,135 - INFO - Predicted IDs: tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  8,  9,  0,  4,  4,  6, 11,
          5,  0,  4,  4, 21,  0,  0,  5,  0,  9,  9,  0,  5, 13,  0,  7,  0, 15,
          0,  0,  4,  4,  0,  0,  0,  0,  0, 23,  0, 13,  0, 10,  9,  

Computing metrics:   2%|█▎                                                          | 1/44 [18:10<13:01:26, 1090.38s/it]

2025-09-12 22:59:57,607 - INFO - Processing sample 2/44
2025-09-12 22:59:57,616 - INFO - Model confidence: 0.8777
2025-09-12 22:59:57,617 - INFO - Computing SHAP values
2025-09-12 22:59:57,618 - INFO - Created background samples with shape torch.Size([5, 90240])
2025-09-12 22:59:57,618 - INFO - Background mean: 0.000013
2025-09-12 22:59:57,619 - INFO - Background std: 0.010005
2025-09-12 22:59:57,657 - INFO - Model output shape: torch.Size([1, 281])
2025-09-12 22:59:57,658 - INFO - Model output mean: 8.835357
2025-09-12 22:59:57,659 - INFO - Model output std: 2.657515
2025-09-12 22:59:57,659 - INFO - Model output sum: 2482.735352
2025-09-12 22:59:57,673 - INFO - Predicted IDs: tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  8,  9,  0,  4,  4,  6, 11,
          5,  0,  4,  4, 21,  0,  0,  5,  0,  9,  0,  0,  5,  0, 13,  0,  7, 15,
         15,  0,  4,  4,  4,  0,  0,  0, 23,  0,  0, 13,  0, 10,  9,  0

...

Computing metrics:  95%|██████████████████████████████████████████████████████████▏  | 42/44 [9:01:33<20:14, 607.18s/it]

2025-09-13 07:43:20,382 - INFO - Processing sample 43/44
2025-09-13 07:43:20,390 - INFO - Model confidence: 0.8482
2025-09-13 07:43:20,390 - INFO - Computing SHAP values
2025-09-13 07:43:20,392 - INFO - Created background samples with shape torch.Size([5, 58720])
2025-09-13 07:43:20,392 - INFO - Background mean: -0.000006
2025-09-13 07:43:20,393 - INFO - Background std: 0.009985
2025-09-13 07:43:19,613 - INFO - Model output shape: torch.Size([1, 183])
2025-09-13 07:43:19,614 - INFO - Model output mean: 7.922424
2025-09-13 07:43:19,615 - INFO - Model output std: 2.530515
2025-09-13 07:43:19,615 - INFO - Model output sum: 1449.803711
2025-09-13 07:43:19,626 - INFO - Predicted IDs: tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  7,  9, 14,  4,  4,  0,  0,
         10,  0,  0,  0,  4,  6,  0,  5,  0,  0,  4,  4, 21,  0,  8,  0,  6,  0,
          0,  4,  6,  0,  0,  5,  0,  0,  0,  4,  0,  0,  0,  8,  0, 

Computing metrics:  98%|███████████████████████████████████████████████████████████▌ | 43/44 [9:10:58<09:54, 594.54s/it]

2025-09-13 07:52:45,435 - INFO - Processing sample 44/44
2025-09-13 07:52:45,443 - INFO - Model confidence: 0.8616
2025-09-13 07:52:45,444 - INFO - Computing SHAP values
2025-09-13 07:52:45,445 - INFO - Created background samples with shape torch.Size([5, 58720])
2025-09-13 07:52:45,445 - INFO - Background mean: 0.000024
2025-09-13 07:52:45,446 - INFO - Background std: 0.009987
2025-09-13 07:52:45,484 - INFO - Model output shape: torch.Size([1, 183])
2025-09-13 07:52:45,485 - INFO - Model output mean: 8.439185
2025-09-13 07:52:45,485 - INFO - Model output std: 2.268975
2025-09-13 07:52:45,486 - INFO - Model output sum: 1544.370850
2025-09-13 07:52:45,498 - INFO - Predicted IDs: tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  7,  9,  0,  0,  4,  0,  0,
         10,  0,  0,  0,  4, 24,  0,  5,  0,  0,  4,  4,  6,  0,  5,  0,  0,  0,
          0,  4,  6,  0,  0,  5,  0,  0,  0,  4,  0, 18,  8,  0,  8,  

Computing metrics: 100%|█████████████████████████████████████████████████████████████| 44/44 [9:20:23<00:00, 764.16s/it]
